In [1]:
import tkinter as tk
from tkinter import ttk, messagebox
import time
import random

In [2]:
class SudokuGUI:
    def __init__(self, master):
        self.master = master
        master.title("Sudoku")
        self.master.resizable(False, False)  # Prevent resizing of the window
        self.board = [[tk.StringVar() for _ in range(9)] for _ in range(9)]
        self.cells = [[None for _ in range(9)] for _ in range(9)]
        self.start_time = None
        self.difficulty_levels = {
            "Easy": 0.3,
            "Medium": 0.5,
            "Hard": 0.7
        }

        # Styling
        self.style = ttk.Style()
        self.style.configure("TButton", font=('Helvetica', 12), padding=6)
        self.style.configure("TEntry", font=('Helvetica', 14), padding=6)
        self.style.configure("TLabel", font=('Helvetica', 16))

        # Sudoku Grid Frame
        self.grid_frame = ttk.Frame(master)
        self.grid_frame.grid(row=0, column=0, padx=10, pady=5)

        # Create grid of cells
        for i in range(9):
            for j in range(9):
                bg_color = '#f0f0f0' if (i // 3 + j // 3) % 2 == 0 else '#cccccc'
                cell = ttk.Entry(self.grid_frame, width=2, font=('Arial', 18), justify='center', textvariable=self.board[i][j], background=bg_color)
                cell.grid(row=i, column=j, stick="nsew", padx=1, pady=1)
                self.cells[i][j] = cell

                # Add borders to create 3x3 grid layout
                if i % 3 == 0:
                    cell.grid(pady=(3, 1))  # Add top border
                if j % 3 == 0:
                    cell.grid(padx=(3, 1))  # Add left border
                if i == 8:
                    cell.grid(pady=(1, 3))  # Add bottom border
                if j == 8:
                    cell.grid(padx=(1, 3))  # Add right border

        # Level Selection and Puzzle Selection
        self.level_var = tk.StringVar(value="Easy")
        level_frame = ttk.Frame(master)
        level_frame.grid(row=0, column=1, padx=10, pady=5, sticky="n")
        ttk.Label(level_frame, text="Level:").grid(row=0, column=0, padx=5, sticky="e")
        self.level_menu = ttk.OptionMenu(level_frame, self.level_var, "Easy", *self.difficulty_levels.keys(), command=self.change_level)
        self.level_menu.grid(row=0, column=1, padx=5, sticky="ew")

        # Puzzle Selection
        puzzle_var = tk.StringVar(value="Puzzle 1")
        puzzle_frame = ttk.Frame(master)
        puzzle_frame.grid(row=1, column=1, padx=10, pady=5, sticky="n")
        ttk.Label(puzzle_frame, text="Puzzle:").grid(row=0, column=0, padx=5, sticky="e")
        puzzle_options = ["Puzzle 1", "Puzzle 2", "Puzzle 3", "Puzzle 4"]
        for i, option in enumerate(puzzle_options):
            ttk.Radiobutton(puzzle_frame, text=option, variable=puzzle_var, value=option, command=self.reset).grid(row=i+1, column=0, padx=5, sticky="w")

        # Timer Label
        self.timer_label = ttk.Label(master, text="Time: 0.000 seconds")
        self.timer_label.grid(row=2, column=1, padx=10, pady=(5, 10), sticky="n")

        # Solve and Reset Buttons
        solve_frame = ttk.Frame(master)
        solve_frame.grid(row=0, column=2, rowspan=3, padx=10, pady=5, sticky="ns")

        ttk.Button(solve_frame, text="Solve (Backtracking)", command=self.solve_backtracking).pack(fill="x", padx=5, pady=5)
        ttk.Button(solve_frame, text="Solve (Arc-3)", command=self.solve_arc3).pack(fill="x", padx=5, pady=5)
        ttk.Button(solve_frame, text="Reset", command=self.reset).pack(fill="x", padx=5, pady=5)




        # Initialize a random game
        self.generate_random_state()

    def change_level(self, level):
        self.generate_random_state()

    def generate_random_state(self):
        difficulty = self.difficulty_levels[self.level_var.get()]
        self.board = [[tk.StringVar() for _ in range(9)] for _ in range(9)]

        # Generate a solved puzzle
        solved = [[(i * 3 + i // 3 + j) % 9 + 1 for j in range(9)] for i in range(9)]
        self.board = [[tk.StringVar(value=str(solved[i][j]) if random.random() < difficulty else "") for j in range(9)] for i in range(9)]

        self.refresh_board()

    def refresh_board(self):
        for i in range(9):
            for j in range(9):
                self.cells[i][j].delete(0, tk.END)
                self.cells[i][j].insert(0, self.board[i][j].get())

    def reset(self):
        self.generate_random_state()

    def solve_backtracking(self):
        self.start_time = time.time()
        if self.solve_sudoku_backtracking():
            elapsed_time = time.time() - self.start_time
            self.timer_label.config(text=f"Time: {elapsed_time:.3f} seconds")
            messagebox.showinfo("Sudoku Solver (Backtracking)", "Puzzle solved!")
            self.refresh_board()  # Refresh the board after solving
        else:
            elapsed_time = time.time() - self.start_time
            self.timer_label.config(text=f"Time: {elapsed_time:.3f} seconds")
            messagebox.showinfo("Sudoku Solver (Backtracking)", "No solution exists. Showing best attempt.")
            self.highlight_conflicts()

    def solve_arc3(self):
        self.start_time = time.time()
        if self.solve_sudoku_arc3():
            elapsed_time = time.time() - self.start_time
            self.timer_label.config(text=f"Time: {elapsed_time:.3f} seconds")
            messagebox.showinfo("Sudoku Solver (Arc-3)", "Puzzle solved!")
            self.refresh_board()  # Refresh the board after solving
        else:
            elapsed_time = time.time() - self.start_time
            self.timer_label.config(text=f"Time: {elapsed_time:.3f} seconds")
            messagebox.showinfo("Sudoku Solver (Arc-3)", "No solution exists. Showing best attempt.")
            self.highlight_conflicts()

    def highlight_conflicts(self):
        for i in range(9):
            for j in range(9):
                if not self.is_valid(i, j, self.board[i][j].get()):
                    self.cells[i][j].config(foreground='red')

    def solve_sudoku_backtracking(self):
        # Backtracking algorithm
        empty = self.find_empty_location()
        if not empty:
            return True
        row, col = empty

        for num in range(1, 10):
            if self.is_valid(row, col, str(num)):
                self.board[row][col].set(str(num))
                if self.solve_sudoku_backtracking():
                    return True
                self.board[row][col].set("")
        return False

    def solve_sudoku_arc3(self):
        # Arc-3 algorithm
        # Initialize the domain for each cell
        domain = {}
        for i in range(9):
            for j in range(9):
                if self.board[i][j].get() == "":
                    domain[(i, j)] = set(str(num) for num in range(1, 10))

        # Define the constraints for each cell
        constraints = {}
        for i in range(9):
            for j in range(9):
                if self.board[i][j].get() == "":
                    constraints[(i, j)] = set()
                    # Add constraints for the row and column
                    for k in range(9):
                        if k != j:
                            constraints[(i, j)].add((i, k))
                        if k != i:
                            constraints[(i, j)].add((k, j))
                    # Add constraints for the 3x3 subgrid
                    start_row, start_col = 3 * (i // 3), 3 * (j // 3)
                    for k in range(3):
                        for l in range(3):
                            if start_row + k != i or start_col + l != j:
                                constraints[(i, j)].add((start_row + k, start_col + l))

        # Perform arc consistency
        queue = list(domain.keys())
        while queue:
            cell_i, cell_j = queue.pop(0)
            for constraint_i, constraint_j in constraints[(cell_i, cell_j)]:
                if (constraint_i, constraint_j) != (cell_i, cell_j):
                    if len(domain.get((constraint_i, constraint_j), set())) == 1:
                        value = next(iter(domain.get((constraint_i, constraint_j), set())))
                        if value in domain.get((cell_i, cell_j), set()):
                            domain[(cell_i, cell_j)].remove(value)
                            if len(domain[(cell_i, cell_j)]) == 0:
                                return False
                            queue.append((cell_i, cell_j))

        # Backtrack with the arc-3 algorithm
        return self.arc3_backtrack(domain)

    def arc3_backtrack(self, domain):
        # Backtracking with the arc-3 algorithm
        empty = self.find_empty_location()
        if not empty:
            return True
        row, col = empty

        for value in domain[(row, col)]:
            if self.is_valid(row, col, value):
                self.board[row][col].set(value)
                if self.solve_sudoku_arc3():
                    return True
                self.board[row][col].set("")
        return False

    def find_empty_location(self):
        for i in range(9):
            for j in range(9):
                if self.board[i][j].get() == "":
                    return i, j
        return None

    def is_valid(self, row, col, num):
        for i in range(9):
            if self.board[i][col].get() == num or self.board[row][i].get() == num:
                return False

        start_row, start_col = 3 * (row // 3), 3 * (col // 3)
        for i in range(3):
            for j in range(3):
                if self.board[start_row + i][start_col + j].get() == num:
                    return False

        return True

In [3]:
def main():
    root = tk.Tk()
    app = SudokuGUI(root)
    root.mainloop()

if __name__ == "__main__":
    main()
